In [3]:
import joblib
import inflection
import pandas as pd

class Fraud:
    
    def __init__(self):
        self.minmaxscaler = joblib.load('./parameters/minmaxscaler_restored.joblib')
        self.onehotencoder = joblib.load('./parameters/onehotencoder_cycle1.joblib')
        
    def data_cleaning(self, df1):
        cols_old = df1.columns.tolist()
        
        snakecase = lambda i: inflection.underscore(i)
        cols_new = list(map(snakecase, cols_old))
        
        df1.columns = cols_new
        
        return df1
    
    def feature_engineering(self, df2):
        # step
        df2['step_days'] = df2['step'].apply(lambda i: i/24)
        df2['step_weeks'] = df2['step'].apply(lambda i: i/(24*7))

        # difference between initial balance before the transaction and new balance after the transaction
        df2['diff_new_old_balance'] = df2['newbalance_orig'] - df2['oldbalance_org']

        # difference between initial balance recipient before the transaction and new balance recipient after the transaction.
        df2['diff_new_old_destiny'] = df2['newbalance_dest'] - df2['oldbalance_dest']

        # name orig and name dest
        df2['name_orig'] = df2['name_orig'].apply(lambda i: i[0])
        df2['name_dest'] = df2['name_dest'].apply(lambda i: i[0])
        
        return df2.drop(columns=['name_orig', 'name_dest', 'step_weeks', 'step_days'], axis=1)

    def data_preparation(self, df3):
        # One Hot Encoding
        df3 = pd.get_dummies(df3)
        #print(df3)
        
        # Rescaling 
        num_columns = ['step', 'oldbalance_org', 'newbalance_orig', 
                       'newbalance_dest', 'diff_new_old_balance', 'diff_new_old_destiny']
        #print(df3[num_columns])
        df3[num_columns] = self.minmaxscaler.transform(df3[num_columns])
        print(df3[num_columns])
        
        # OneHotEncoder
        #df3 = self.onehotencoder.transform(df3)
        
        # selected columns
        final_columns_selected = ['step', 'oldbalance_org', 'newbalance_orig'
                                  , 'newbalance_dest', 'diff_new_old_balance', 'diff_new_old_destiny'] #'type_TRANSFER']
        return df3[final_columns_selected]
    
    def get_prediction(self, model, original_data, test_data):
        pred = model.predict(test_data)
        if pred == 1 : 
            y_prob = model.predict_proba(test_data)[:, 1]
        else : 
            y_prob = model.predict_proba(test_data)[:, 0]

        original_data['prediction'] = pred
        original_data['class_probability'] = y_prob
        
        return original_data.to_json(orient="records", date_format="iso")

In [4]:
import joblib
import pandas as pd
#from churn.Churn import Churn
from flask import Flask, request, Response
#from fraud.Fraud import Fraud

# loading model
model = joblib.load('./models/model_cycle1.joblib')

# initialize API
app = Flask(__name__)

@app.route('/fraud/predict', methods=['POST'])
def churn_predict():
    try:
        test_json = request.get_json()
    
        if test_json: # there is data
            if isinstance(test_json, dict): # unique example
                test_raw = pd.DataFrame(test_json, index=[0])
                
            else: # multiple example
                test_raw = pd.DataFrame(test_json, columns=test_json[0].keys())
                
            # Instantiate Rossmann class
            pipeline = Fraud()
            
            # data cleaning
            df1 = pipeline.data_cleaning(test_raw)
            
            # feature engineering
            df2 = pipeline.feature_engineering(df1)
            
            # data preparation
            df3 = pipeline.data_preparation(df2)
            
            # prediction
            df_response = pipeline.get_prediction(model, test_raw, df3)
            
            return df_response
            
            
        else:
            return Response('{}', status=200, mimetype='application/json')
    except Exception as e:
        return Response(f'{{"error": "{str(e)}"}}', status=500, mimetype='application/json')

if __name__ == '__main__':
    app.run('0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.39:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Oct/2025 08:56:00] "POST /fraud/predict HTTP/1.1" 200 -


       step  oldbalance_org  newbalance_orig  newbalance_dest  \
0  0.252362             0.0              0.0              0.0   

   diff_new_old_balance  diff_new_old_destiny  
0                   0.0                   0.0  


In [51]:
import joblib
import pandas as pd

# โหลด MinMaxScaler ที่บันทึกไว้
mm = joblib.load('./parameters/minmaxscaler_cycle1.joblib')

# ดึงค่าที่สำคัญออกมา
scaler_params = {
    'feature_names': mm.feature_names_in_.tolist(),
    'data_min': mm.data_min_.tolist(),
    'data_max': mm.data_max_.tolist(),
    'scale': mm.scale_.tolist(),
    'min': mm.min_.tolist(),
}

# แปลงเป็น DataFrame
df_params = pd.DataFrame(scaler_params)

# บันทึกเป็น CSV
df_params.to_csv('./parameters/minmaxscaler_cycle1_params.csv', index=False)

# หรือบันทึกเป็น JSON
df_params.to_json('./parameters/minmaxscaler_cycle1_params.json', orient='records', indent=4)

print("Scaler parameters saved!")


Scaler parameters saved!


In [54]:
import json
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load JSON
with open('./parameters/minmaxscaler_cycle1_params.json', 'r') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Create and fill MinMaxScaler
scaler = MinMaxScaler()

# Manually set attributes
scaler.feature_names_in_ = df['feature_names'].to_numpy()
scaler.data_min_ = df['data_min'].to_numpy()
scaler.data_max_ = df['data_max'].to_numpy()
scaler.scale_ = df['scale'].to_numpy()
scaler.min_ = df['min'].to_numpy()
scaler.data_range_ = scaler.data_max_ - scaler.data_min_
scaler.n_features_in_ = len(scaler.feature_names_in_)

# Save as .joblib
joblib.dump(scaler, './parameters/minmaxscaler_restored.joblib')

print("✅ MinMaxScaler restored and saved as joblib.")

✅ MinMaxScaler restored and saved as joblib.


In [ ]:
y_prob = model.predict_proba(X_test)

AttributeError: type object 'Fraud' has no attribute 'predict_proba'

In [9]:
df_raw = pd.read_csv("C:/Users/artit/Downloads/fraud_0.1origbase.csv") #.iloc[5, :-2]
print(df_raw.head(5))

   step      type     amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0   283   CASH_IN  210329.84  C1159819632     3778062.79      3988392.64   
1   132  CASH_OUT  215489.19  C1372369468       21518.00            0.00   
2   355     DEBIT    4431.05  C1059822709       20674.00        16242.95   
3   135  CASH_OUT  214026.20  C1464960643       46909.73            0.00   
4   381  CASH_OUT    8858.45   C831134427           0.00            0.00   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  C1218876138      1519266.60      1308936.76        0               0  
1   C467105520      6345756.55      6794954.89        0               0  
2    C76588246        80876.56        85307.61        0               0  
3  C1059379810     13467450.36     13681476.56        0               0  
4   C579876929      1667180.58      1676039.03        0               0  


In [13]:
def __init__(self):
        self.minmaxscaler = joblib.load('./parameters/minmaxscaler_cycle1.joblib')
        self.onehotencoder = joblib.load('./parameters/onehotencoder_cycle1.joblib')
        
def data_cleaning(self, df1):
        cols_old = df1.columns.tolist()
        
        snakecase = lambda i: inflection.underscore(i)
        cols_new = list(map(snakecase, cols_old))
        
        df1.columns = cols_new
        
        return df1

In [34]:
from sklearn.preprocessing   import MinMaxScaler

mm = MinMaxScaler()
mm.fit(X_params_cs, y_temp)
mm
#joblib.dump(mm, './parameters/minmaxscaler_cycle1.joblib')

NameError: name 'X_params_cs' is not defined

In [15]:
fraud=Fraud()
print(type(fraud.minmaxscaler))       # >>> <class 'sklearn.preprocessing._data.MinMaxScaler'>
print(type(fraud.onehotencoder))  

<class 'sklearn.preprocessing._data.MinMaxScaler'>
<class 'category_encoders.one_hot.OneHotEncoder'>


In [19]:
df_clean = fraud.data_cleaning(df_raw)
# 4. ดูผลลัพธ์
print(df_clean.columns)

Index(['step', 'type', 'amount', 'name_orig', 'oldbalance_org',
       'newbalance_orig', 'name_dest', 'oldbalance_dest', 'newbalance_dest',
       'is_fraud', 'is_flagged_fraud', 'step_days', 'step_weeks',
       'diff_new_old_balance', 'diff_new_old_destiny'],
      dtype='object')


In [20]:
df_feature = fraud.feature_engineering(df_clean)
print(df_feature.columns)

Index(['step', 'type', 'amount', 'oldbalance_org', 'newbalance_orig',
       'oldbalance_dest', 'newbalance_dest', 'is_fraud', 'is_flagged_fraud',
       'diff_new_old_balance', 'diff_new_old_destiny'],
      dtype='object')


In [44]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df3 = pd.get_dummies(df_feature)
        
# Rescaling 
num_columns = ['step', 'oldbalance_org', 'newbalance_orig', 
                       'newbalance_dest', 'diff_new_old_balance', 'diff_new_old_destiny']
df3[num_columns] = fraud.minmaxscaler.transform(df3[num_columns])
        
# OneHotEncoder
#df3 = self.onehotencoder.transform(df3)
        
# selected columns
final_columns_selected = ['step', 'oldbalance_org', 'newbalance_orig', 'newbalance_dest', 
                                  'diff_new_old_balance', 'diff_new_old_destiny'] #'type_TRANSFER']

NameError: name 'df_feature' is not defined

In [56]:
print(fraud.minmaxscaler.feature_names_in_)

['step' 'oldbalance_org' 'newbalance_orig' 'newbalance_dest'
 'diff_new_old_balance' 'diff_new_old_destiny']


In [60]:
print(pd.__version__)

2.3.0


In [10]:
# loading test dataset
df10 = pd.read_csv("C:/Users/artit/Downloads/fraud_0.1origbase.csv").iloc[5, :-2]

In [11]:
df10.head()

step                     208
type                 CASH_IN
amount             256440.86
nameOrig         C1001269496
oldbalanceOrg          554.0
Name: 5, dtype: object

In [12]:
# convert Dataframe to json
data = df10.to_json()

In [13]:
data

'{"step":208,"type":"CASH_IN","amount":256440.86,"nameOrig":"C1001269496","oldbalanceOrg":554.0,"newbalanceOrig":256994.86,"nameDest":"C1503528288","oldbalanceDest":0.0,"newbalanceDest":0.0}'